In [ ]:
%%capture
!pip config set global.index-url https://pypi.tuna.tsinghua.edu.cn/simple
!pip install modelscope gpustat
#!modelscope download --model Fengshenbang/Erlangshen-RoBERTa-330M-Sentiment
!pip install langchain-ollama ollama

In [27]:
!wget https://gh.ddlc.top/https://github.com/ollama/ollama/releases/download/v0.13.2/ollama-linux-amd64.tgz
!sudo tar zxvf ollama-linux-amd64.tgz -C /usr

--2025-12-09 08:40:21--  https://gh.ddlc.top/https://github.com/ollama/ollama/releases/download/v0.13.2/ollama-linux-amd64.tgz
Resolving gh.ddlc.top (gh.ddlc.top)... 104.19.89.51
Connecting to gh.ddlc.top (gh.ddlc.top)|104.19.89.51|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1960719094 (1.8G) [application/octet-stream]
Saving to: ‘ollama-linux-amd64.tgz.1’

ollama-linux-amd64. 100%[===================>]   1.83G  14.4MB/s    in 2m 17s  

utime(ollama-linux-amd64.tgz.1): No such file or directory
2025-12-09 08:42:39 (13.7 MB/s) - ‘ollama-linux-amd64.tgz.1’ saved [1960719094/1960719094]

tar (child): ollama-linux-amd64.tgz: Cannot open: No such file or directory
tar (child): Error is not recoverable: exiting now
tar: Child returned status 2
tar: Error is not recoverable: exiting now
time=2025-12-09T08:42:40.950Z level=INFO source=routes.go:1544 msg="server config" env="map[CUDA_VISIBLE_DEVICES:GPU-d4cbbe92-05ec-a8ba-e2f1-b7992c8e7c93 GGML_VK_VISIBLE_DEVICES: 

### run codes in terminal  
nohup ollama serve &  
ollama pull gemma3:270m

In [29]:
from langchain_ollama import OllamaLLM

# Initialize the Ollama model (e.g., llama3.2, gemma2, phi3, etc.)
llm = OllamaLLM(
    model="gemma3:270m",        # or "gemma2", "mistral", "phi3", etc.
    temperature=0.7,
    base_url="http://localhost:11434",  # default, change if needed
)

# List of prompts you want to run in batch
prompts = [
    "Explain quantum computing in simple terms.",
    "Write a haiku about artificial intelligence.",
    "Translate this to French: 'The quick brown fox jumps over the lazy dog.'",
    "Summarize the theory of relativity in one paragraph.",
    "Tell me a short joke about programmers.",
]

print("Running batch inference...\n")

# This invokes all prompts in parallel (batch mode)
responses = llm.batch(prompts, config={"max_concurrent": 10})  # optional concurrency limit

# Print results
for prompt, response in zip(prompts, responses):
    print(f"Prompt: {prompt}")
    print(f"Response: {response.strip()}\n")
    print("-" * 80)

Running batch inference...

Prompt: Explain quantum computing in simple terms.
Response: Imagine you have a box filled with LEGO bricks. You can only put one brick at a time. You can't put all the bricks in one go.

Quantum computing is like having a super-powered box that can **store and manipulate information in a way that is impossible for regular computers.**

Here's the breakdown:

*   **Quantum mechanics is the study of the very small:** It's about the weird world of particles like electrons and photons.
*   **Quantum computers use "qubits":** These are like dimmer switches. They can be in multiple states at the same time, allowing them to explore many possibilities simultaneously.
*   **They can solve problems that are impossible for regular computers:** This is where quantum computing comes in. It could help us solve problems that are currently too hard for even the most powerful supercomputers.

**Think of it like this:**

*   **Regular computers:**  They do calculations based

In [31]:
from langchain_ollama import OllamaLLM
from langchain_core.prompts import PromptTemplate
import json
from typing import Literal

# Choose one of these (make sure it's pulled with `ollama pull <name>`)
MODEL = "gemma3:270m"          # ← super fast + accurate for sentiment
# MODEL = "llama3.2:3b"      # also excellent
# MODEL = "gemma2:9b-it"     # highest accuracy

llm = OllamaLLM(
    model=MODEL,
    temperature=0.0,         # deterministic = consistent sentiment labels
    format="json",           # forces valid JSON output (works great with phi3/gemma2/llama3+)
)

# Zero-shot prompt that forces clean JSON output
sentiment_prompt = PromptTemplate.from_template(
    """Analyze the sentiment of the following customer review.

Return ONLY a valid JSON object with exactly these three keys:
{{"sentiment": "positive" | "negative" | "neutral", 
  "confidence": a number between 0.0 and 1.0,
  "explanation": short reason in 1 sentence}}

Text: "{text}"

JSON:"""
)

# Example dataset (replace with your own list, DataFrame column, etc.)
reviews = [
    "I absolutely love this product! Best purchase ever.",
    "It's okay, nothing special. Does the job.",
    "Terrible customer service and the item broke in half.",
    "Super fast shipping and great quality!",
    "Not bad, but the color is different from the photo.",
    "Worst experience ever. Never buying again.",
    "Meh. Expected more for the price.",
]

print(f"Running batch sentiment analysis on {len(reviews)} reviews using {MODEL}...\n")

# This runs all prompts in parallel (true batching)
chain = sentiment_prompt | llm
results = chain.batch([{"text": r} for r in reviews], config={"max_concurrency": 20})

# Parse and display clean results
print("Results:\n" + "="*60)
for i, raw_output in enumerate(results):
    try:
        # Clean common artifacts (some models add ```json or extra text)
        json_str = raw_output.strip()
        if json_str.startswith("```json"):
            json_str = json_str[7:-3]
        elif json_str.startswith("```"):
            json_str = json_str[3:-3]
        
        data = json.loads(json_str)
        
        sentiment = data.get("sentiment", "unknown").upper()
        confidence = data.get("confidence", 0.0)
        explanation = data.get("explanation", "")
        
        print(f"{i+1:2d}. [{sentiment:8}] (conf: {confidence:.2f}) → {reviews[i][:60]}...")
        if explanation:
            print(f"     ↳ {explanation}\n")
            
    except json.JSONDecodeError as e:
        print(f"{i+1:2d}. [PARSE ERROR] Raw output: {raw_output}\n")

print("Batch sentiment analysis completed!")

Running batch sentiment analysis on 7 reviews using gemma3:270m...

Results:
 1. [POSITIVE] (conf: 0.90) → I absolutely love this product! Best purchase ever....
 2. [POSITIVE] (conf: 0.90) → It's okay, nothing special. Does the job....
 3. [NEGATIVE] (conf: 0.95) → Terrible customer service and the item broke in half....
     ↳ The customer service was extremely negative, with a broken item and a negative rating.

 4. [POSITIVE] (conf: 0.80) → Super fast shipping and great quality!...
     ↳ The customer is very satisfied with the fast shipping and quality of the product.

 5. [POSITIVE] (conf: 0.80) → Not bad, but the color is different from the photo....
 6. [NEGATIVE] (conf: 0.95) → Worst experience ever. Never buying again....
     ↳ The customer expresses strong dissatisfaction with the experience, indicating a negative sentiment.

 7. [NEGATIVE] (conf: 0.90) → Meh. Expected more for the price....
     ↳ The customer was disappointed with the price and the product quality.

Batch